<a href="https://colab.research.google.com/github/SharWarr/ML_Projects/blob/main/Ecommerce_Churn_Project/Notebook_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook --no-browser"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [ ]:
# Spark environment
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [ ]:
MAX_MEMORY = "14G"

spark = SparkSession \
    .builder \
    .appName("demo") \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

spark

23/02/20 04:48:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
df=spark.read.parquet("Cleaned_df_final_parquet.parquet")

In [ ]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- brand_new: string (nullable = true)
 |-- target: integer (nullable = true)



In [ ]:
df = df.drop("user_id","product_id","category_id","brand","user_session")

In [ ]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 6, 12, 18, 24 ],inputCol="Hour", outputCol="Hour_binned")
df_buck = bucketizer.setHandleInvalid("keep").transform(df)

df_buck.show()

+----------+--------------------+------+-----------+--------------------+----+---------+------+-----------+
|event_type|       category_code| price| category_1|          category_2|Hour|brand_new|target|Hour_binned|
+----------+--------------------+------+-----------+--------------------+----+---------+------+-----------+
|      view|electronics.smart...|341.74|electronics|                null|  10|   xiaomi|     0|        1.0|
|      view|         no category| 36.04|no category|                null|  10| no brand|     0|        1.0|
|      view|         no category| 34.11|no category|                null|  10|   Others|     0|        1.0|
|      view|         no category| 63.06|no category|                null|  13|   Others|     0|        2.0|
|      view|         no category|341.91|no category|                null|  12| no brand|     0|        2.0|
|      view|         no category|362.34|no category|                null|  12| no brand|     0|        2.0|
|      view|         no cate

In [ ]:
df_buck = df_buck.fillna(value ='no category',subset =['category_2'])

In [ ]:
df_buck.show()

+----------+--------------------+------+-----------+--------------------+----+---------+------+-----------+
|event_type|       category_code| price| category_1|          category_2|Hour|brand_new|target|Hour_binned|
+----------+--------------------+------+-----------+--------------------+----+---------+------+-----------+
|      view|electronics.smart...|341.74|electronics|         no category|  10|   xiaomi|     0|        1.0|
|      view|         no category| 36.04|no category|         no category|  10| no brand|     0|        1.0|
|      view|         no category| 34.11|no category|         no category|  10|   Others|     0|        1.0|
|      view|         no category| 63.06|no category|         no category|  13|   Others|     0|        2.0|
|      view|         no category|341.91|no category|         no category|  12| no brand|     0|        2.0|
|      view|         no category|362.34|no category|         no category|  12| no brand|     0|        2.0|
|      view|         no cate

In [ ]:
from pyspark.sql.types import IntegerType,FloatType
df_buck = df_buck.withColumn("Hour_binned", df_buck["Hour_binned"].cast(IntegerType()))
df_buck = df_buck.withColumn("price", df_buck["price"].cast(FloatType()))

In [ ]:
df_buck.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- price: float (nullable = true)
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = false)
 |-- Hour: integer (nullable = true)
 |-- brand_new: string (nullable = true)
 |-- target: integer (nullable = true)
 |-- Hour_binned: integer (nullable = true)



In [ ]:
df_buck = df_buck.drop("Hour")

In [ ]:
# cast the click column to interger data type. 

from pyspark.sql.types import IntegerType

df_buck = df_buck.withColumn("y", df["target"].cast(IntegerType()))

In [ ]:
#A user defined function for sampling the data. 

from math import floor
from pyspark.sql.functions import rand
from pyspark.sql.functions import col

def stratifiedSample(df, N, labelCol="y"):
    ctx = df.groupby(labelCol).count()
    ctx = ctx.withColumn('frac', col("count") / df.count())
    frac = ctx.select("y", "frac").rdd.collectAsMap()
    pos = int(floor(frac[1] * N))
    neg = int(floor(frac[0] * N))
    posDF = df.filter(col(labelCol) == 1).orderBy(rand()).limit(pos)
    negDF = df.filter(col(labelCol) == 0).orderBy(rand()).limit(neg)
    return posDF.unionAll(negDF)

In [ ]:
# use the function created to sample the 5M rows from the complete dataset. 

xdf = stratifiedSample(df_buck, 5_000_000)

In [ ]:
xdf.groupby("y").count().show()

+---+-------+
|  y|  count|
+---+-------+
|  1|1668207|
|  0|3331792|
+---+-------+



In [ ]:
xdf.groupby("y").count().withColumn("frac", col("count") / xdf.count()).show()

+---+-------+-------------------+
|  y|  count|               frac|
+---+-------+-------------------+
|  1|1668207|0.33364146672829337|
|  0|3331792| 0.6663585332717067|
+---+-------+-------------------+



In [ ]:
xdf.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- price: float (nullable = true)
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = false)
 |-- brand_new: string (nullable = true)
 |-- target: integer (nullable = true)
 |-- Hour_binned: integer (nullable = true)
 |-- y: integer (nullable = true)



In [ ]:
xdf.groupBy("y").count().show()

+---+-------+
|  y|  count|
+---+-------+
|  1|1668207|
|  0|3331792|
+---+-------+



In [ ]:
xdf.groupBy("target").count().show()

+------+-------+
|target|  count|
+------+-------+
|     1|1668207|
|     0|3331792|
+------+-------+



In [ ]:
xdf = xdf.drop("target")

In [ ]:
xdf.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- price: float (nullable = true)
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = false)
 |-- brand_new: string (nullable = true)
 |-- Hour_binned: integer (nullable = true)
 |-- y: integer (nullable = true)



In [ ]:
xdf.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = true)
 |-- brand_new: string (nullable = true)
 |-- Hour_binned: integer (nullable = true)
 |-- y: integer (nullable = true)



In [ ]:
# onehot encoding 
#import the onehot encoder
from pyspark.ml.feature import OneHotEncoderEstimator
#create the encoder object
ohe = OneHotEncoderEstimator(inputCols=['Hour_binned'], outputCols=['Hour_binned_posEnc'])
#fit the obejct to the dataframe
oh_encoder = ohe.fit(df_buck)
#tranform the dataframe, by adding the 
encoded = oh_encoder.transform(df_buck)

encoded.show()

+----------+--------------------+------+-----------+--------------------+---------+------+-----------+---+------------------+
|event_type|       category_code| price| category_1|          category_2|brand_new|target|Hour_binned|  y|Hour_binned_posEnc|
+----------+--------------------+------+-----------+--------------------+---------+------+-----------+---+------------------+
|      view|electronics.smart...|341.74|electronics|         no category|   xiaomi|     0|          1|  0|     (3,[1],[1.0])|
|      view|         no category| 36.04|no category|         no category| no brand|     0|          1|  0|     (3,[1],[1.0])|
|      view|         no category| 34.11|no category|         no category|   Others|     0|          1|  0|     (3,[1],[1.0])|
|      view|         no category| 63.06|no category|         no category|   Others|     0|          2|  0|     (3,[2],[1.0])|
|      view|         no category|341.91|no category|         no category| no brand|     0|          2|  0|     (3,[2],

In [ ]:
# import the string indexer
from pyspark.ml.feature import StringIndexer
#similar to the one hot encoder, create a string indexer object and fit it to the dataframe, use the fitted object to transform the dataset.
si = StringIndexer(inputCol='event_type', outputCol='event_type_ix')
encoded = si.fit(encoded).transform(encoded)
#use the output of the sting indexer as an input to the onehot encoder. 
ohe = OneHotEncoderEstimator(inputCols=['event_type_ix'], outputCols=['event_type_ixEnc'])
oh_encoder = ohe.fit(encoded)
encoded = oh_encoder.transform(encoded)

encoded.select("event_type_ixEnc").show()

+----------------+
|event_type_ixEnc|
+----------------+
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
|   (2,[0],[1.0])|
+----------------+
only showing top 20 rows



In [ ]:
#similar to the one hot encoder, create a string indexer object and fit it to the dataframe, use the fitted object to transform the dataset.
si = StringIndexer(inputCol='category_1', outputCol='category_1_ix')
encoded = si.fit(encoded).transform(encoded)
#use the output of the sting indexer as an input to the onehot encoder. 
ohe = OneHotEncoderEstimator(inputCols=['category_1_ix'], outputCols=['category_1_ixEnc'])
oh_encoder = ohe.fit(encoded)
encoded = oh_encoder.transform(encoded)

encoded.select("category_1_ixEnc").show()

+----------------+
|category_1_ixEnc|
+----------------+
|  (13,[0],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[1],[1.0])|
|  (13,[2],[1.0])|
|  (13,[2],[1.0])|
|  (13,[2],[1.0])|
|  (13,[2],[1.0])|
|  (13,[2],[1.0])|
|  (13,[2],[1.0])|
|  (13,[0],[1.0])|
|  (13,[0],[1.0])|
|  (13,[0],[1.0])|
|  (13,[0],[1.0])|
+----------------+
only showing top 20 rows



In [ ]:
si = StringIndexer(inputCol='category_2', outputCol='category_2_ix')
encoded = si.fit(encoded).transform(encoded)
#use the output of the sting indexer as an input to the onehot encoder. 
ohe = OneHotEncoderEstimator(inputCols=['category_2_ix'], outputCols=['category_2_ixEnc'])
oh_encoder = ohe.fit(encoded)
encoded = oh_encoder.transform(encoded)

encoded.select("category_2_ixEnc").show()

+----------------+
|category_2_ixEnc|
+----------------+
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[3],[1.0])|
|  (85,[3],[1.0])|
|  (85,[3],[1.0])|
|  (85,[3],[1.0])|
|  (85,[3],[1.0])|
|  (85,[3],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
|  (85,[0],[1.0])|
+----------------+
only showing top 20 rows



In [ ]:
si = StringIndexer(inputCol='brand_new', outputCol='brand_new_ix')
encoded = si.fit(encoded).transform(encoded)
#use the output of the sting indexer as an input to the onehot encoder. 
ohe = OneHotEncoderEstimator(inputCols=['brand_new_ix'], outputCols=['brand_new_ixEnc'])
oh_encoder = ohe.fit(encoded)
encoded = oh_encoder.transform(encoded)

encoded.select("brand_new_ixEnc").show()

+---------------+
|brand_new_ixEnc|
+---------------+
| (20,[4],[1.0])|
| (20,[1],[1.0])|
| (20,[0],[1.0])|
| (20,[0],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[0],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[1],[1.0])|
| (20,[5],[1.0])|
| (20,[4],[1.0])|
| (20,[5],[1.0])|
| (20,[5],[1.0])|
+---------------+
only showing top 20 rows



In [ ]:
encoded.columns

['event_type',
 'category_code',
 'price',
 'category_1',
 'category_2',
 'brand_new',
 'target',
 'Hour_binned',
 'y',
 'Hour_binned_posEnc',
 'event_type_ix',
 'event_type_ixEnc',
 'category_1_ix',
 'category_1_ixEnc',
 'category_2_i',
 'category_2_ix',
 'category_2_ixEnc']

In [ ]:
#import the vector assembler 
from pyspark.ml.feature import VectorAssembler

#create the assembler object
assembler = VectorAssembler(inputCols=['Hour_binned_posEnc',
 'event_type_ixEnc',
 'category_1_ixEnc',
 'category_2_ixEnc',
 'brand_new_ixEnc',
 'price'], outputCol="features")
#transform the data frame using the assembler object. 
encoded = assembler.transform(encoded)
encoded.select("features").show()

+--------------------+
|            features|
+--------------------+
|(124,[1,3,5,18,10...|
|(124,[1,3,6,18,10...|
|(124,[1,3,6,18,10...|
|(124,[2,3,6,18,10...|
|(124,[2,3,6,18,10...|
|(124,[2,3,6,18,10...|
|(124,[2,3,6,18,10...|
|(124,[2,3,6,18,10...|
|(124,[2,3,6,18,10...|
|(124,[2,3,6,18,10...|
|(124,[2,3,7,21,10...|
|(124,[2,3,7,21,10...|
|(124,[2,3,7,21,10...|
|(124,[2,3,7,21,10...|
|(124,[2,3,7,21,10...|
|(124,[2,3,7,21,10...|
|(124,[2,3,5,18,10...|
|(124,[1,3,5,18,10...|
|(124,[0,3,5,18,10...|
|(124,[0,3,5,18,10...|
+--------------------+
only showing top 20 rows



In [ ]:
encoded.select("features","y").show()

+--------------------+---+
|            features|  y|
+--------------------+---+
|(124,[1,3,5,18,10...|  0|
|(124,[1,3,6,18,10...|  0|
|(124,[1,3,6,18,10...|  0|
|(124,[2,3,6,18,10...|  0|
|(124,[2,3,6,18,10...|  0|
|(124,[2,3,6,18,10...|  0|
|(124,[2,3,6,18,10...|  0|
|(124,[2,3,6,18,10...|  0|
|(124,[2,3,6,18,10...|  0|
|(124,[2,3,6,18,10...|  0|
|(124,[2,3,7,21,10...|  0|
|(124,[2,3,7,21,10...|  0|
|(124,[2,3,7,21,10...|  0|
|(124,[2,3,7,21,10...|  0|
|(124,[2,3,7,21,10...|  0|
|(124,[2,3,7,21,10...|  0|
|(124,[2,3,5,18,10...|  0|
|(124,[1,3,5,18,10...|  0|
|(124,[0,3,5,18,10...|  0|
|(124,[0,3,5,18,10...|  0|
+--------------------+---+
only showing top 20 rows



In [ ]:
model_df = encoded.select("features","y")

In [ ]:
#split the data 
training_df,test_df=model_df.randomSplit([0.75,0.25])

#import the logistic regression 
from pyspark.ml.classification import LogisticRegression

#Apply the logistic regression model
log_reg=LogisticRegression(labelCol='y').fit(training_df)

#Training Results
train_results=log_reg.evaluate(training_df).predictions
train_results.filter(train_results['y']==1).filter(train_results['prediction']==1).select(['y','prediction','probability']).show(10,False)

[Stage 325:>                                                        (0 + 4) / 7]23/02/20 05:27:32 WARN MemoryStore: Not enough space to cache rdd_679_0 in memory! (computed 551.5 MB so far)
23/02/20 05:27:32 WARN BlockManager: Persisting block rdd_679_0 to disk instead.
23/02/20 05:27:32 WARN MemoryStore: Not enough space to cache rdd_679_2 in memory! (computed 551.5 MB so far)
23/02/20 05:27:32 WARN BlockManager: Persisting block rdd_679_2 to disk instead.
23/02/20 05:27:32 WARN MemoryStore: Not enough space to cache rdd_679_3 in memory! (computed 552.0 MB so far)
23/02/20 05:27:32 WARN BlockManager: Persisting block rdd_679_3 to disk instead.
[Stage 555:>                                                        (0 + 1) / 1]

+---+----------+----------------------------------------+
|y  |prediction|probability                             |
+---+----------+----------------------------------------+
|1  |1.0       |[0.24812254061668726,0.7518774593833127]|
|1  |1.0       |[0.2481425740571454,0.7518574259428547] |
|1  |1.0       |[0.24827348599466106,0.751726514005339] |
|1  |1.0       |[0.24827348599466106,0.751726514005339] |
|1  |1.0       |[0.2483496499459881,0.7516503500540119] |
|1  |1.0       |[0.24855550439164803,0.7514444956083519]|
|1  |1.0       |[0.24902244112909075,0.7509775588709092]|
|1  |1.0       |[0.24914294974882775,0.7508570502511723]|
|1  |1.0       |[0.24917777048457182,0.7508222295154282]|
|1  |1.0       |[0.2491898246277536,0.7508101753722464] |
+---+----------+----------------------------------------+
only showing top 10 rows



In [ ]:
accuracy = train_results.filter(train_results.y == train_results.prediction).count() / float(train_results.count())
print("Accuracy : ",accuracy)


[Stage 564:================================================>        (6 + 1) / 7]

Accuracy :  0.693945872751744


In [ ]:
tp = train_results.filter((col("y") == 1) & (col("prediction") == 1)).count()
fp = train_results.filter((col("y") == 0) & (col("prediction") == 1)).count()
fn = train_results.filter((col("y") == 1) & (col("prediction") == 0)).count()

[Stage 598:=========>       (4 + 3) / 7][Stage 602:>                (0 + 1) / 7]23/02/20 06:13:12 WARN MemoryStore: Not enough space to cache rdd_1294_0 in memory! (computed 364.2 MB so far)
[Stage 598:=========>       (4 + 3) / 7][Stage 604:>                (0 + 1) / 7]23/02/20 06:13:30 WARN MemoryStore: Not enough space to cache rdd_1294_0 in memory! (computed 364.2 MB so far)
[Stage 614:=========>       (4 + 3) / 7][Stage 622:>                (0 + 1) / 7]23/02/20 06:16:39 WARN MemoryStore: Not enough space to cache rdd_1294_0 in memory! (computed 364.2 MB so far)
[Stage 614:=========>       (4 + 3) / 7][Stage 624:>                (0 + 1) / 7]23/02/20 06:16:59 WARN MemoryStore: Not enough space to cache rdd_1294_0 in memory! (computed 364.2 MB so far)


In [ ]:
# Calculate the precision and recall
precision = tp / (tp + fp)
recall = tp / (tp + fn)

# Print the precision and recall
print("Precision: ", precision)
print("Recall: ", recall)

Precision:  0.8576116040223289
Recall:  0.09938306315266973


In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

In [ ]:
# Create vector assembler
# Load data
#pipeline

assembler = VectorAssembler(inputCols=['Hour_binned_posEnc',
 'event_type_ixEnc',
 'category_1_ixEnc',
 'category_2_ixEnc',
 'brand_new_ixEnc',
 'price'], outputCol='features_pipeline')

# Create logistic regression model
lr = LogisticRegression(featuresCol='features_pipeline', labelCol="y")

# Create pipeline
pipeline = Pipeline(stages=[assembler, lr])

In [ ]:
# Define grid of hyperparameters to test
param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .addGrid(lr.maxIter, [10, 100]) \
    .build()

In [ ]:
# Define cross-validator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=param_grid,
                    evaluator=BinaryClassificationEvaluator(),
                    numFolds=5)

In [ ]:
# Split data into training and testing sets
training, testing = encoded.randomSplit([0.7, 0.3], seed=42)

# Fit cross-validator to training data
cv_model = cv.fit(training)

# Use the best model to make predictions on the testing set
predictions = cv_model.transform(testing)

# Evaluate the performance of the model
evaluator = BinaryClassificationEvaluator()
#area_under_curve = evaluator.evaluate(predictions)

23/02/20 06:21:04 WARN CacheManager: Asked to cache already cached data.
23/02/20 06:21:04 WARN CacheManager: Asked to cache already cached data.
[Stage 661:>                (0 + 0) / 2][Stage 662:>                (0 + 4) / 7]23/02/20 06:21:18 WARN MemoryStore: Not enough space to cache rdd_1200_3 in memory! (computed 1486.3 MB so far)
23/02/20 06:21:18 WARN MemoryStore: Not enough space to cache rdd_1200_2 in memory! (computed 1485.2 MB so far)
23/02/20 06:21:28 WARN MemoryStore: Not enough space to cache rdd_1445_0 in memory! (computed 232.5 MB so far)
23/02/20 06:21:28 WARN BlockManager: Persisting block rdd_1445_0 to disk instead.
23/02/20 06:21:28 WARN MemoryStore: Not enough space to cache rdd_1445_2 in memory! (computed 226.2 MB so far)
23/02/20 06:21:28 WARN BlockManager: Persisting block rdd_1445_2 to disk instead.
23/02/20 06:21:29 WARN MemoryStore: Not enough space to cache rdd_1445_3 in memory! (computed 232.2 MB so far)
23/02/20 06:21:29 WARN BlockManager: Persisting block

IllegalArgumentException: 'Field "label" does not exist.\nAvailable fields: event_type, category_code, price, category_1, category_2, brand_new, target, Hour_binned, y, Hour_binned_posEnc, event_type_ix, event_type_ixEnc, category_1_ix, category_1_ixEnc, category_2_i, category_2_ix, category_2_ixEnc, brand_new_ix, brand_new_ixEnc, features, CrossValidator_b6f7896f094f_rand, features_pipeline, rawPrediction, probability, prediction'

In [ ]:
area_under_curve = evaluator.evaluate(predictions)

[Stage 723:>                (0 + 0) / 2][Stage 724:>                (0 + 4) / 7]23/02/20 06:24:54 WARN MemoryStore: Not enough space to cache rdd_1200_2 in memory! (computed 1485.2 MB so far)
23/02/20 06:25:02 WARN MemoryStore: Not enough space to cache rdd_1596_3 in memory! (computed 102.9 MB so far)
23/02/20 06:25:02 WARN BlockManager: Persisting block rdd_1596_3 to disk instead.
23/02/20 06:25:02 WARN MemoryStore: Not enough space to cache rdd_1596_0 in memory! (computed 154.8 MB so far)
23/02/20 06:25:02 WARN BlockManager: Persisting block rdd_1596_0 to disk instead.
23/02/20 06:25:02 WARN MemoryStore: Not enough space to cache rdd_1596_1 in memory! (computed 154.5 MB so far)
23/02/20 06:25:02 WARN BlockManager: Persisting block rdd_1596_1 to disk instead.
[Stage 724:==>              (1 + 4) / 7][Stage 726:>                (0 + 0) / 7]23/02/20 06:25:20 WARN MemoryStore: Not enough space to cache rdd_1200_4 in memory! (computed 1485.6 MB so far)
[Stage 724:=========>       (4 + 3) /

In [ ]:
print(evaluator)

[Stage 1016:=============>  (6 + 1) / 7][Stage 1018:>               (0 + 3) / 7]

NameError: name 'evaluator' is not defined

[Stage 1019:>               (0 + 0) / 2][Stage 1020:>               (0 + 4) / 7]23/02/20 06:39:11 WARN MemoryStore: Not enough space to cache rdd_1200_2 in memory! (computed 1485.2 MB so far)
23/02/20 06:39:11 WARN MemoryStore: Not enough space to cache rdd_1200_3 in memory! (computed 1486.3 MB so far)
23/02/20 06:39:20 WARN MemoryStore: Not enough space to cache rdd_2269_2 in memory! (computed 226.2 MB so far)
23/02/20 06:39:20 WARN BlockManager: Persisting block rdd_2269_2 to disk instead.
23/02/20 06:39:20 WARN MemoryStore: Not enough space to cache rdd_2269_0 in memory! (computed 232.5 MB so far)
23/02/20 06:39:20 WARN BlockManager: Persisting block rdd_2269_0 to disk instead.
23/02/20 06:39:21 WARN MemoryStore: Not enough space to cache rdd_2269_3 in memory! (computed 232.2 MB so far)
23/02/20 06:39:21 WARN BlockManager: Persisting block rdd_2269_3 to disk instead.
23/02/20 06:39:21 WARN MemoryStore: Not enough space to cache rdd_2269_1 in memory! (computed 233.4 MB so far)
23/02/